In [1]:
import os
#파이썬 인터프리터 제어
import sys
#날짜관련
import datetime
# Python HTTP 관련
import requests
# 시간관련
import time
# 문자열 처리관련(정규식)
import re

# 웹 자동화(컨트롤) 관련
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

# 데이터 스크래핑 관련
from bs4 import BeautifulSoup
#beautifulsoup 파싱형식
import lxml

#데이터 처리,분석
import pandas as pd

#엑셀처리
import openpyxl
from openpyxl.styles import NamedStyle,Border, Side,Alignment



sURL = "https://www.youtube.com/watch?v=2wNTys-rPWs"

sURL = "https://www.youtube.com/watch?v=x6Y8S1msF14"
#sURL = "https://www.youtube.com/watch?v=gwMa6gpoE9I"
#https://www.youtube.com/watch?v=j45mjlHNnKs #토크멘터리
#https://www.youtube.com/watch?v=x6Y8S1msF14
#https://www.youtube.com/watch?v=Py5P4Ufv16Q
#https://www.youtube.com/watch?v=gwMa6gpoE9I #방탄 큰거

In [2]:
#셀레니움 옵션 설정
ops = webdriver.ChromeOptions()
#ops.add_argument('headless')
#창 사이즈
ops.add_argument('window-size=1920x1080')
#창 위치
ops.add_argument('window-position=500,0')
#ops.add_argument(
#"user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
ops.add_argument("lang=ko_KR")  # 한국어
ops.add_experimental_option("prefs", {
"profile.default_content_setting_values.automatic_downloads": True,
"directory_upgrade": True,
"safebrowsing.enabled": True})

In [3]:
with webdriver.Chrome(chrome_options=ops) as mydr:
    mydr.implicitly_wait(20)

    # 사이트 접속
    mydr.get(sURL)
    try:
        # 특정 오브젝트 로딩까지 wait
        element_present = EC.presence_of_element_located((By.ID,'more'))
        WebDriverWait(mydr, 20).until(element_present)
#/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/ytd-comments

    except TimeoutException:
        pass
     # 스크롤을 위한 사이즈(화면 1/3크기) 저장
    last_page_height = mydr.execute_script("return document.documentElement.scrollHeight")/3
    # 저장한 크기만큼 스크롤 (댓글창 로딩위해)
    mydr.execute_script(f"window.scrollTo(0, {last_page_height});")
    # 더보기 클릭 (description 부분)
    oBtn_More = mydr.find_element_by_xpath('//paper-button[@id="more"]')
    oBtn_More.click()
    
    for i in range(0,10):
        # 스크롤을 위한 사이즈(화면 1/3크기) 저장
        last_page_height = mydr.execute_script("return document.documentElement.scrollHeight")
        # 저장한 크기만큼 스크롤 (댓글창 로딩위해)
        mydr.execute_script(f"window.scrollTo(0, {last_page_height});")
        #if mydr.find_elements_by_xpath('//html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[4]/div[1]/div/ytd-comments'):
        if EC.presence_of_element_located((By.XPATH , '//ytd-comment-renderer[@id="comment"]')):
            break
      

    # 댓글창 로딩 wait
    print("wait for comments to load ...")
    WebDriverWait(mydr, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comments ytd-comment-simplebox-renderer"))
    )
    


    #댓글창 스크롤 시작
    last_size = 0
    nEndCount = 0
    nCount = 20
    for i in range(0,nCount):
        # 지금까지 로딩된 댓글 전체 리스트
        lst_element = mydr.find_elements_by_xpath('//ytd-comment-renderer[@id="comment"]')
        # 댓글 리스트 중 마지막 코멘트 위치로
        last_element = lst_element[-1]
        # 마지막 코멘트로 이동(추가 댓글 로딩을 위해)
        mydr.execute_script("arguments[0].scrollIntoView(true);", last_element);
        time.sleep(0.5)
        new_size = len(lst_element)
        
        # 추가 로딩 했는데 댓글 갯수가 같으면 Count+1
        if new_size == last_size:
            nEndCount+= 1
            print(last_size)
            print(new_size)
        else:
            nEndCount = 0
        # Count가 2이상이면 끝
        if nEndCount >1:
            print("break: "+ str(i))
            break
        last_size = new_size

    print('코멘트 중 자세히 부분 클릭')
    lst_more = mydr.find_elements_by_xpath('//paper-button[@id="more"]')
    for data in lst_more:
        if '자세히' in data.text:
            #mydr.execute_script("arguments[0].scrollIntoView(true);", data);
            mydr.execute_script("arguments[0].click();", data);
            #data.send_keys(Keys.Endter)

    print('코멘트 클릭 끝\n')

    sScripts = mydr.page_source

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


wait for comments to load ...
20
20
20
20
break: 4
코멘트 중 자세히 부분 클릭
코멘트 클릭 끝



In [4]:
#데이터 스크래핑 시작
parse = BeautifulSoup(sScripts,'lxml')

# 해당 비디오 정보 부분
sYoutube_Title = parse.find('meta', {'property': 'og:title'})['content']
sYoutube_URL = parse.find('meta', {'property': 'og:url'})['content']
sYoutube_Image = parse.find('meta', {'property': 'og:image'})['content']

sYoutube_Count = parse.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text
rx = re.compile(r'([조회수 ,회])')
sYoutube_Count = rx.sub('',sYoutube_Count)

sYoutube_Date = parse.find('div',{'id':'date'}).find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
rx = re.compile(r'([ ])')
sYoutube_Date = rx.sub('',sYoutube_Date)

# 한글 단위 -> 숫자형식으로 (엑셀 활용시 편의성위해)
# 한글이 더좋으면 말씀해주세요
nUnit_Like = 1
nUnit_Dislike = 1
sYoutube_Like = parse.find('yt-formatted-string',{'aria-label':re.compile(r'좋아요')}).text
rx = re.compile(r'([천 만])')
if '천' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 1000
elif '만' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 10000
nYoutube_Like = int(float(sYoutube_Like) * nUnit_Like)


sYoutube_Dislike = parse.find('yt-formatted-string', {'aria-label': re.compile(r'싫어요')}).text
if '천' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 1000
elif '만' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 10000
nYoutube_Dislike = int(float(sYoutube_Dislike)*nUnit_Dislike)


#sYoutube_Discription = parse.find('meta', {'property': 'og:description'})['content']



print('제목: ' + sYoutube_Title)
print('URL: ' + sYoutube_URL)
print('Image_URL: ' + sYoutube_Image)
print('조회수: ' + sYoutube_Count)
print('게시일: ' + sYoutube_Date)
print('좋아요: ' + str(nYoutube_Like))
print('싫어요: ' + str(nYoutube_Dislike))


pdVideo_Info = pd.DataFrame({'sYoutube_Title':[sYoutube_Title],
                             'sYoutube_URL':[sYoutube_URL],
                             'sYoutube_Image':[sYoutube_Image],
                             'sYoutube_Count':[sYoutube_Count],
                             'sYoutube_Date':[sYoutube_Date],
                             'nYoutube_Like':[nYoutube_Like],
                             'nYoutube_Dislike':[nYoutube_Dislike]
                             })
pdVideo_Info

제목: 🔥혈압주의 🔥 X같은 회사, 편안한 직장생활을 위한 슬기로운 꿀팁 전수!ㅣ스튜디오 콜 ep.06(마지막회)
URL: https://www.youtube.com/watch?v=x6Y8S1msF14
Image_URL: https://i.ytimg.com/vi/x6Y8S1msF14/maxresdefault.jpg
조회수: 715784
게시일: 2020.2.6.
좋아요: 220
싫어요: 9


,sYoutube_Title,sYoutube_URL,sYoutube_Image,sYoutube_Count,sYoutube_Date,nYoutube_Like,nYoutube_Dislike
0,"🔥혈압주의 🔥 X같은 회사, 편안한 직장생활을 위한 슬기로운 꿀팁 전수!ㅣ스튜디오 ...",https://www.youtube.com/watch?v=x6Y8S1msF14,https://i.ytimg.com/vi/x6Y8S1msF14/maxresdefau...,715784,2020.2.6.,220,9


In [5]:
# 채널 정보 시작
sYoutube_Channel_Nm = parse.find('ytd-channel-name',{'id':'channel-name'}).find('a').text

sYoutube_Channel_Sub_Count = parse.find('yt-formatted-string',{'id':'owner-sub-count'}).text
# 한글 -> 숫자 처리
rx = re.compile(r'([구독자 만명])')
nUnit = 1
if '천' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 1000
elif '만' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 10000
nYoutube_Channel_Sub_Count = int(float(sYoutube_Channel_Sub_Count) * nUnit)

sYoutube_Discription = parse.find('div', {'id': 'description'}).text
sYoutube_Category = parse.find(text=re.compile(r'^카테고리')).find_parent().find_parent().find_next_sibling().find('a', {'class': 'yt-simple-endpoint style-scope yt-formatted-string'}).text
lst_Youtube_tag = parse.find_all('meta',{'property':'og:video:tag'})
sYoutube_tag = ', '.join(data['content'] for data in lst_Youtube_tag)

print('채널 명: ' + sYoutube_Channel_Nm)
print('구독자:' + str(nYoutube_Channel_Sub_Count))
print('------------------------------------------------')
print('Description: ' +sYoutube_Discription)
print('------------------------------------------------')
print('카테고리 :' + sYoutube_Category)     
print('------------------------------------------------')
print('tag: ' + sYoutube_tag)
print('------------------------------------------------')

pdChannel_Info = pd.DataFrame({'sYoutube_Channel_Nm': [sYoutube_Channel_Nm],
                               'nYoutube_Channel_Sub_Count': [nYoutube_Channel_Sub_Count],
                               'sYoutube_Description': [sYoutube_Discription],
                               'sYoutube_Category': [sYoutube_Category],
                               'sYoutube_tag': [sYoutube_tag]
                               })
pdChannel_Info

채널 명: 스튜디오 룰루랄라- studio lululala
구독자:701000
------------------------------------------------
Description: 흔한 직장 내 노답 빌런 모음;;
김슬기의 야무진 회사 생활 노하우?!
체력 고갈 K-직장인에게 추천합니다

무엇이든 제작하는 [스튜디오 콜]
1화 보러가기 👉 https://www.youtube.com/watch?v=gEw33...
 
#스튜디오콜 #StudioCall #스타트업 #광고회사
#공감 #웹드라마 #스튜디오룰루랄라 #초월홍삼
------------------------------------------------
카테고리 :엔터테인먼트
------------------------------------------------
tag: 스튜디오룰루랄라, 웹시트콤, 스튜디오콜, 광고회사, 한국시트콤, 오피스, 직장인공감, 현실공감, 직장인웹드, 직장인, 공감, 스타트업, 퇴사, 회사현실, Studiocall, 초월홍삼, 김슬기, 송가인, 현대자동차
------------------------------------------------


,sYoutube_Channel_Nm,nYoutube_Channel_Sub_Count,sYoutube_Description,sYoutube_Category,sYoutube_tag
0,스튜디오 룰루랄라- studio lululala,701000,흔한 직장 내 노답 빌런 모음;;\n김슬기의 야무진 회사 생활 노하우?!\n체력 고...,엔터테인먼트,"스튜디오룰루랄라, 웹시트콤, 스튜디오콜, 광고회사, 한국시트콤, 오피스, 직장인공감..."


In [6]:
# 코멘트 부분 시작

pd_lstComments = pd.DataFrame({'sComments_Author': [],
                           'sComments_Time': [],
                           'sComments_text': [],
                           'nComments_Like': []
                           })
lst_comments = parse.find_all('ytd-comment-thread-renderer',{'class':'style-scope ytd-item-section-renderer'})
        
#print('======================================================')
for data in lst_comments:
    sAuthor = data.find('a',{'id':'author-text'}).text.strip()
    sTime = data.find('yt-formatted-string', {'class': re.compile('published-time-text')}).text.strip()
    sComment = data.find('yt-formatted-string',{'id':'content-text'}).text.replace('\n',' ').strip()
    sLike = data.find('span',{'id':'vote-count-middle'}).text.strip()
    rx = re.compile(r'([천 만])')
    nUnit_Like = 1
    if '천' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 1000
    elif '만' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 10000
    nLike = int(float(sLike) * nUnit_Like)
    #print(sAuthor)
    #print(sTime)
    #print(sComment)
    #print("좋아요: " + str(nLike))
    #print('======================================================')

    pdComments = pd.DataFrame({'sComments_Author': [sAuthor],
                           'sComments_Time': [sTime],
                           'sComments_text': [sComment],
                           'nComments_Like': [nLike]
                           })
    pd_lstComments = pd_lstComments.append(pdComments)
pd_lstComments.index = range(len(pd_lstComments))

In [7]:
sFileNm = 'Youtube_Data.xlsx'
#wb = openpyxl.Workbook()

#m_border = Border(right=Side(style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))
ali_center = Alignment(horizontal="center", vertical="center",shrink_to_fit=True)
ali = Alignment(wrap_text=False, shrink_to_fit=True) 

#리스트부분 시작
# 파일 없으면
if not os.path.exists(sFileNm):
    wb = openpyxl.Workbook()
    ws_list = wb.active
    ws_list.title = '리스트'
    #ws.merge_cells('A2:A3')
    ws_list.merge_cells('A2:A3')
    ws_list.merge_cells('B2:D2')
    ws_list.merge_cells('E2:J2')
    ws_list.merge_cells('K2:K3')
    
    #ws.merge_cells('K2:K3')
    ws_list['A2'] = '인덱스'
    ws_list['B2'] = '채널정보'
    ws_list['E2'] = '콘텐츠정보'
    ws_list['B3'] = '채널명'
    ws_list['C3'] = '구독자수'
    ws_list['D3'] = '카테고리'
    ws_list['E3'] = '타이틀'
    ws_list['F3'] = '주소'
    ws_list['G3'] = '조회수'
    ws_list['H3'] = '날짜'
    ws_list['I3'] = '좋아요'
    ws_list['J3'] = '싫어요'
    ws_list['K2'] = '업데이트 날짜(데이터)'

    
    for data in ws_list['A2':'K3']:
        for v in data:
            #v.border = m_border
            v.alignment = ali_center
    
    nIndex = 4
    bNewChk = True
#파일 있으면
else:
    wb=openpyxl.load_workbook(sFileNm)
    sheet_nm =wb.sheetnames
    ws_list = wb[sheet_nm[0]]
    print ('max row: ' +str(ws_list.max_row))
    
    nIndex = 4
    bNewChk = True
    for i in range(4,ws_list.max_row+5):
        tmp_URL =ws_list.cell(column = 6, row = i).value
        if tmp_URL :
            if tmp_URL == pdVideo_Info['sYoutube_URL'][0]:
                print ('동일데이터 OK, index: '+str(i))
                nIndex = i
                bNewChk = False
                break
            else: 
                print ('동일데이터 x, index: '+str(i))
        else:
            bNewChk =True
            nIndex = i
            break

print('nIndex :' +str(nIndex))

        
ws_list.cell(column=1,row=nIndex,value=nIndex-3)
ws_list.cell(column=2,row=nIndex,value=pdChannel_Info['sYoutube_Channel_Nm'][0])
ws_list.cell(column=3,row=nIndex,value=pdChannel_Info['nYoutube_Channel_Sub_Count'][0])
ws_list.cell(column=4,row=nIndex,value=pdChannel_Info['sYoutube_Category'][0])

ws_list.cell(column=5,row=nIndex,value=pdVideo_Info['sYoutube_Title'][0])
ws_list.cell(column=6,row=nIndex,value=pdVideo_Info['sYoutube_URL'][0])
ws_list.cell(column=7,row=nIndex,value=pdVideo_Info['sYoutube_Count'][0])
ws_list.cell(column=8,row=nIndex,value=pdVideo_Info['sYoutube_Date'][0])
ws_list.cell(column=9,row=nIndex,value=pdVideo_Info['nYoutube_Like'][0])
ws_list.cell(column=10,row=nIndex,value=pdVideo_Info['nYoutube_Dislike'][0])
ws_list.cell(column=11,row=nIndex,value=datetime.datetime.now().strftime('%Y.%m.%d'))



# width 조절
for col in ws_list.columns:
    max_length = 0
    column = col[0].column # Get the column name
    for cell in col:
        if cell.row >3 :
            cell.alignment = ali
        if cell.coordinate in ws_list.merged_cells: # not check merge_cells
            continue
        try: # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1
    ws_list.column_dimensions[column].width = adjusted_width



# 추가 디테일 시트부분 시작
if bNewChk :
    print('새데이터')
    ws = wb.create_sheet(pdChannel_Info['sYoutube_Channel_Nm'][0]+'_'+str(nIndex-3))
else:
    print('기존데이터')
    ws = wb[pdChannel_Info['sYoutube_Channel_Nm'][0]+'_'+str(nIndex-3)]
    
ws.merge_cells('A2:C2')
ws.merge_cells('D2:L2')
ws.merge_cells('M2:P2')

ws['A2'] = '채널정보'
ws['D2'] = '콘텐츠정보'
ws['M2'] = '댓글정보'
ws['A3'] = '채널명'
ws['B3'] = '구독자수'
ws['C3'] = '카테고리'
ws['D3'] = '타이틀'
ws['E3'] = '주소'
ws['F3'] = '썸네일'
ws['G3'] = '디스크립션'
ws['H3'] = '조회수'
ws['I3'] = '날짜'
ws['J3'] = '좋아요'
ws['K3'] = '싫어요'
ws['L3'] = '숨김태그'
ws['M3'] = '아이디'
ws['N3'] = '댓글작성일'
ws['O3'] = '댓글'
ws['P3'] = '댓글좋아요'

for data in ws['A2':'P3']:
    for v in data:
        #v.border = m_border
        v.alignment = ali_center

nIndex_sheet = 4
ws.cell(column=1,row=nIndex_sheet,value=pdChannel_Info['sYoutube_Channel_Nm'][0]+'@@@')
ws.cell(column=2,row=nIndex_sheet,value=pdChannel_Info['nYoutube_Channel_Sub_Count'][0])
ws.cell(column=3,row=nIndex_sheet,value=pdChannel_Info['sYoutube_Category'][0])

ws.cell(column=4,row=nIndex_sheet,value=pdVideo_Info['sYoutube_Title'][0])
ws.cell(column=5,row=nIndex_sheet,value=pdVideo_Info['sYoutube_URL'][0])
ws.cell(column=6,row=nIndex_sheet,value=pdVideo_Info['sYoutube_Image'][0])
ws.cell(column=7,row=nIndex_sheet,value=pdChannel_Info['sYoutube_Description'][0])
ws.cell(column=8,row=nIndex_sheet,value=pdVideo_Info['sYoutube_Count'][0])
ws.cell(column=9,row=nIndex_sheet,value=pdVideo_Info['sYoutube_Date'][0])
ws.cell(column=10,row=nIndex_sheet,value=pdVideo_Info['nYoutube_Like'][0])
ws.cell(column=11,row=nIndex_sheet,value=pdVideo_Info['nYoutube_Dislike'][0])
ws.cell(column=12,row=nIndex_sheet,value=pdChannel_Info['sYoutube_tag'][0])
for i in range(len(pd_lstComments.values)):
    ws.cell(column=13,row=nIndex_sheet+i,value=pd_lstComments.values[i][0])
    ws.cell(column=14,row=nIndex_sheet+i,value=pd_lstComments.values[i][1])
    ws.cell(column=15,row=nIndex_sheet+i,value=pd_lstComments.values[i][2])
    ws.cell(column=16,row=nIndex_sheet+i,value=pd_lstComments.values[i][3])

# width 조절
for col in ws.columns:
    max_length = 0
    column = col[0].column # Get the column name
    for cell in col:
        if cell.row >3 :
            cell.alignment = ali
        if cell.coordinate in ws.merged_cells: # not check merge_cells
            continue
        try: # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1
    ws.column_dimensions[column].width = adjusted_width

wb.save(filename = sFileNm)
wb.close




max row: 4
4
동일데이터 x, index: 4
5
nIndex :5
새데이터


<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x000001BB4460A128>>